Kendra Robbins 

OSM Week 6 

Simplex Algorithm Lab 

In [22]:
import numpy as np

# Problem 1-6

In [49]:
class simplex: 
    
    def __init__(self,c,A,b):
    
        #attributes 
        self.c = c
        self.A = A 
        self.b = b 

        self.n = len(self.c) #dimension of domain 
        self.m = len(self.b) #dimension of target space
    
        if np.any(self.A @ np.zeros(len(self.c)) > self.b):
            raise ValueError("Not feasible at the origin")
            
        basic = np.arange(self.n, self.n + self.m) # initial basic variables
        nonbasic = np.arange(0, self.n)            # initial nonbasic variables
            
        self.basic = basic
        self.nonbasic=nonbasic
        
        self.L = np.concatenate((basic, nonbasic))
        self.T = self.init_tableau()
        self.basic = self.L[:self.m]
        self.nonbasic = self.L[self.m:]
        
    #a method that creates the first tableau    
    def init_tableau(self):
        '''Creates the initial tableau'''
        c_bar = np.hstack((self.c, np.zeros(self.m))) 
        A_bar = np.hstack((self.A, np.identity(self.m)))
        T_toprow = np.hstack((0, -1 * c_bar, 1))
        T_bottomrow = np.column_stack((self.b, A_bar, np.zeros(self.m)))
        T = np.vstack((T_toprow, T_bottomrow))
        return T
    
    #determine the pivot row and column using bland's rule
    def findpivot(self):

        j = np.argwhere(self.T[0, 1:-1] < 0)[0][0] + 1    # pivot column
        col_j = self.T[1:, j].copy()
        
        if np.all(col_j <= 0):
            raise ValueError("The problem is unbounded.") # terminate algorithm -- no optimal solution
        
        non_pos = col_j <= 0                              # find non-positive coefficients in column j
        col_0 = self.T[1:, 0].copy()
        col_j[non_pos] = np.nan                           # set these entries to NA in copy of column j
        i = np.nanargmin(col_0 / col_j) + 1               # pivot row
        
        return i, j
    
    def pivot(self):
        '''Perform a pivot and update list of basic/nonbasic variables and tableau.'''
        i, j = self.findpivot()                   # get pivot row and column
        
        temp = self.L[i - 1]                     # swap entering and leaving variables in index list
        self.L[i - 1] = self.L[self.m + j - 1]
        self.L[self.m + j - 1] = temp
        
        self.T[i] /= self.T[i, j]                # update tableau
        row_i = self.T[i]
        for k, row in enumerate(self.T):
            if k == i:
                pass
            else:
                mult = -row[j]
                self.T[k] = row_i * mult + row
                
                
    def solve(self):
        '''Call pivot() until a solution is found or problem is determined to be unbounded.'''
        
        while np.any(self.T[0] < 0):
            self.pivot()
        basic_dict = {self.basic[i]: round(self.T[i+1, 0], 2) for i in range(self.m)}
        nonbasic_dict = {self.nonbasic[i]: 0 for i in range(self.n)}
        
        return self.T[0, 0], basic_dict, nonbasic_dict
            
            

In [50]:
b = np.array([1,2])
A = np.array([[3,4],[0,6]])
c = np.array([0,1])
b2 = np.array([-1,2])

A =simplex(c,A,b)
A.basic
A.b
A.solve()

(0.25, {1: 0.25, 3: 0.5}, {0: 0, 2: 0})

# Problem 7

In [70]:
data = np.load('productMix.npz')
a = data['A']
p = data['p']
m = data['m']
d = data['d']

In [71]:
A = np.row_stack([a, np.eye((4))])
b = np.concatenate([m, d])

In [73]:
simplex(p, A, b).solve()

(7453.596491228071,
 {1: 6.19, 3: 1.79, 6: 0.97, 0: 10.0, 8: 13.81, 2: 12.0, 10: 8.21},
 {7: 0, 4: 0, 9: 0, 5: 0})